In [1]:
import pandas as pd

In [34]:
"""
Cryptocurrency Pair Selection and Cointegration Pipeline

This module provides a complete pipeline for:
1. Loading and filtering crypto data
2. Computing correlation matrices
3. Identifying cointegrated pairs
4. Selecting top pairs per rolling window
5. Building feature matrices with spreads and statistics
"""

import os
import glob
import pandas as pd
import numpy as np
from collections import Counter
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from statsmodels.tsa.stattools import adfuller


# ============================================================================
# 1. DATA LOADING AND FILTERING
# ============================================================================

def load_and_filter(folder, start="2024-05-01 00:00:00", end="2025-05-01 00:00:00", 
                    file_pattern="*_1m_bin_futures.parquet", number_of_observations=None):
    """
    Load 1-minute crypto parquet files with full data coverage,
    filter by datetime range, and compute log prices & returns.
    
    Args:
        folder (str): Path to folder containing parquet files
        start (str): Start timestamp (default: "2024-05-01 00:00:00")
        end (str): End timestamp (default: "2025-05-01 00:00:00")
        file_pattern (str): Glob pattern for files to load
    
    Returns:
        dict: {symbol: DataFrame with log prices and returns}
    """
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)
    data = {}

    for f in glob.glob(os.path.join(folder, file_pattern)):
        sym = os.path.basename(f).replace(f"{file_pattern[1:]}", "").replace("USDT", "")
        df = pd.read_parquet(f)

        # Convert to datetime if needed
        if "datetime" in df.columns:
            df["datetime"] = pd.to_datetime(df["datetime"])
        else:
            df["datetime"] = df.index
            df["datetime"] = pd.to_datetime(df["datetime"])
            df = df.reset_index(drop=True)

        # Filter by the specified time window
        df = df[(df["datetime"] >= start) & (df["datetime"] <= end)]
        df = df.set_index("datetime").sort_index()
        if number_of_observations and len(df) < number_of_observations:
            print(f"Not full data for {sym}, only {len(df)} rows, starting from {df.index[0]} to {df.index[-1]} ")
        # Log prices and returns
        df["close"] = np.log(df["close"])
        df["open"] = np.log(df["open"])
        df["high"] = np.log(df["high"])
        df["low"] = np.log(df["low"])
        df["log_return"] = df["close"].diff()

        data[sym] = df
        print(f"Loaded {sym}, {len(df)} rows")

    return data


# ============================================================================
# 2. CORRELATION ANALYSIS
# ============================================================================

def compute_correlation_matrix(crypto_data):
    """
    Compute correlation matrix of closing prices for all cryptos.
    
    Args:
        crypto_data (dict): {symbol: DataFrame}
    
    Returns:
        pd.DataFrame: Correlation matrix with symbols as index and columns
    """
    symbols = list(crypto_data.keys())
    close_prices = pd.DataFrame({sym: crypto_data[sym]["close"] for sym in symbols})
    corr_matrix = close_prices.corr()
    return corr_matrix


def find_high_correlation_pairs(crypto_data, correlation_matrix, threshold=0.85):
    """
    Select pairs with correlation above threshold.
    
    Args:
        crypto_data (dict): {symbol: DataFrame}
        correlation_matrix (pd.DataFrame): Correlation matrix
        threshold (float): Minimum absolute correlation (default: 0.85)
    
    Returns:
        list: List of tuples (sym1, sym2) with high correlation
    """
    high_corr_pairs = []
    symbols = list(crypto_data.keys())
    
    for i in range(len(symbols)):
        for j in range(i + 1, len(symbols)):
            sym1, sym2 = symbols[i], symbols[j]
            corr = correlation_matrix.loc[sym1, sym2]
            if abs(corr) > threshold:
                high_corr_pairs.append((sym1, sym2))
                print(f"High correlation: {sym1} & {sym2} = {corr:.2f}")
    
    return high_corr_pairs


# ============================================================================
# 3. COINTEGRATION TESTING
# ============================================================================

def rolling_cointegration(y, x, window=4320, adf_pval=0.05):
    """
    Rolling Engle–Granger cointegration test with beta estimation.
    
    Args:
        y (pd.Series): First price series (index: datetime)
        x (pd.Series): Second price series (index: datetime)
        window (int): Rolling window size (default: 4320 = 3 days of 1-min bars)
        adf_pval (float): ADF test p-value threshold (default: 0.05)
    
    Returns:
        pd.DataFrame: Cointegration results with columns:
            - start: Window start timestamp
            - end: Window end timestamp
            - alpha: Intercept from OLS regression
            - beta: Slope from OLS regression
            - adf_p: ADF test p-value
            - cointegrated: Boolean cointegration status
            - correlation: Pearson correlation in window
    """
    y, x = y.align(x, join="inner")
    y, x = y.sort_index(), x.sort_index()

    results = []
    step = window // 3  # 1/3 overlap
    timestamps = y.index

    for i in range(0, len(timestamps) - window, step):
        start_time = timestamps[i]
        end_time = timestamps[i + window - 1]

        y_win = y.loc[start_time:end_time]
        x_win = x.loc[start_time:end_time]

        if len(y_win) < window or y_win.isna().any() or x_win.isna().any():
            continue

        model = OLS(y_win, add_constant(x_win)).fit()
        alpha, beta = model.params

        residuals = y_win - model.predict(add_constant(x_win))
        adf_p = adfuller(residuals)[1]
        corr = y_win.corr(x_win)

        results.append({
            "start": start_time,
            "end": end_time,
            "alpha": alpha,
            "beta": beta,
            "adf_p": adf_p,
            "cointegrated": adf_p <= adf_pval,
            "correlation": corr
        })

    return pd.DataFrame(results)


def prepare_all_pairs(crypto_data, list_of_pairs, window=4320, adf_pval=0.05):
    """
    Run rolling cointegration test on all pairs.
    
    Args:
        crypto_data (dict): {symbol: DataFrame}
        list_of_pairs (list): List of (sym1, sym2) tuples
        window (int): Rolling window size
        adf_pval (float): ADF p-value threshold
    
    Returns:
        dict: {(sym1, sym2): cointegration_results_df}
    """
    pair_df = {}
    
    for i, (sym1, sym2) in enumerate(list_of_pairs, 1):
        print(f"Processing pair: {sym1}, {sym2}. {i} of {len(list_of_pairs)}")
        
        y_ohlc = crypto_data[sym1]
        x_ohlc = crypto_data[sym2]

        # Align close prices
        df_close = pd.concat([y_ohlc["close"], x_ohlc["close"]], axis=1, join="inner").dropna()
        y_aligned, x_aligned = df_close.iloc[:, 0], df_close.iloc[:, 1]

        # Rolling cointegration
        coint_df = rolling_cointegration(y_aligned, x_aligned, window=window, adf_pval=adf_pval)
        pair_df[(sym1, sym2)] = coint_df
        filename = f"{sym1}_{sym2}_hl_futures_window_cointegration.csv"
        coint_df.to_csv(filename, index=False)
        print(f"Saved features for pair {sym1}-{sym2} to {filename}")

    return pair_df


# ============================================================================
# 4. PAIR SELECTION (TOP-K PER WINDOW)
# ============================================================================

def select_top_pairs_per_window(coint_df, top_k=5):
    """
    Select top-K cointegrated pairs per rolling window based on correlation.
    
    Args:
        coint_df (dict): {(sym1, sym2): cointegration_results_df}
        top_k (int): Number of top pairs to keep per window (default: 5)
    
    Returns:
        dict: {(start, end): [(pair, corr, beta, alpha, adf_p), ...]}
    """
    top_pairs_per_window = {}
    
    # Collect all cointegrated pairs per window
    for pair, df in coint_df.items():
        for _, row in df.iterrows():
            window_key = (row["start"], row["end"])
            if window_key not in top_pairs_per_window:
                top_pairs_per_window[window_key] = []
            if row["cointegrated"]:
                top_pairs_per_window[window_key].append((
                    pair, row["correlation"], row["beta"], row["alpha"], row["adf_p"]
                ))
    
    # Keep only top-K by absolute correlation
    for window_key, pairs in top_pairs_per_window.items():
        pairs.sort(key=lambda x: abs(x[1]), reverse=True)
        top_pairs_per_window[window_key] = pairs[:top_k]
    
    return top_pairs_per_window


def count_pair_occurrences(top_pairs_per_window):
    """
    Count how many times each pair appears in top-K across all windows.
    
    Args:
        top_pairs_per_window (dict): Output from select_top_pairs_per_window
    
    Returns:
        Counter: {(sym1, sym2): count}
    """
    pair_counter = Counter()
    for pairs in top_pairs_per_window.values():
        for pair_info in pairs:
            pair = pair_info[0]
            pair_counter[pair] += 1
    return pair_counter


# ============================================================================
# 5. FEATURE BUILDING
# ============================================================================

def build_full_features(crypto_data, top_pairs_per_window):
    """
    Build a comprehensive feature matrix with:
    - Log prices for all symbols
    - Spreads for cointegrated pairs (window-specific beta/alpha)
    - Beta, alpha, ADF p-value, and correlation for each pair per window
    
    Args:
        crypto_data (dict): {symbol: DataFrame}
        top_pairs_per_window (dict): Output from select_top_pairs_per_window
    
    Returns:
        pd.DataFrame: Feature matrix with timestamp index
    """
    # Collect all unique symbols from top pairs
    all_symbols = sorted({
        sym
        for pairs in top_pairs_per_window.values()
        for pair_info in pairs
        for sym in pair_info[0]
    })

    # Initialize base DataFrame with full timestamp index
    first_sym = all_symbols[0]
    full_df = pd.DataFrame(index=crypto_data[first_sym].index)
    full_df.index.name = "timestamp"

    # Add log-price columns for each symbol
    for sym in all_symbols:
        df = crypto_data[sym][["close"]].rename(columns={"close": f"{sym}_close"})
        full_df = full_df.join(df, how="left")

    # For each window and cointegrated pair, compute spreads and statistics
    for (start, end), pairs in top_pairs_per_window.items():
        mask = (full_df.index >= start) & (full_df.index <= end)

        for pair_info in pairs:
            (sym1, sym2), corr, beta, alpha, adf_p = pair_info

            spread_col = f"{sym1}_{sym2}_spread"
            beta_col = f"{sym1}_{sym2}_beta"
            alpha_col = f"{sym1}_{sym2}_alpha"
            adf_col = f"{sym1}_{sym2}_adf_p"
            corr_col = f"{sym1}_{sym2}_corr"

            # Initialize columns if they don't exist
            for col in [spread_col, beta_col, alpha_col, adf_col, corr_col]:
                if col not in full_df.columns:
                    full_df[col] = np.nan

            # Compute spread for this window
            y = full_df.loc[mask, f"{sym1}_close"]
            x = full_df.loc[mask, f"{sym2}_close"]

            full_df.loc[mask, spread_col] = y - (alpha + beta * x)
            full_df.loc[mask, beta_col] = beta
            full_df.loc[mask, alpha_col] = alpha
            full_df.loc[mask, adf_col] = adf_p
            full_df.loc[mask, corr_col] = corr
    full_df["timestamp"] = full_df.index
    full_df = full_df.reset_index(drop=True)
    return full_df


# ============================================================================
# 6. MAIN PIPELINE
# ============================================================================

def run_pair_selection_pipeline(folder, start_date, end_date, number_of_observations=221110,
                                corr_threshold=0.85, window=4320, 
                                adf_pval=0.05, top_k=5, 
                                file_pattern="*_1m_bin_futures.parquet",
                                output_file=None):
    """
    Complete pipeline: load data → correlations → cointegration → feature building
    
    Args:
        folder (str): Path to parquet files
        start_date (str): Start timestamp
        end_date (str): End timestamp
        corr_threshold (float): Correlation threshold (default: 0.85)
        window (int): Rolling window size (default: 4320)
        adf_pval (float): ADF p-value threshold (default: 0.05)
        top_k (int): Number of top pairs per window (default: 5)
        file_pattern (str): Glob pattern for files
        output_file (str): Optional path to save CSV output
    
    Returns:
        tuple: (crypto_data, correlation_matrix, high_corr_pairs, 
                coint_df, top_pairs_per_window, full_features_df)
    """
    print("=" * 80)
    print("STEP 1: Loading and filtering data")
    print("=" * 80)
    crypto_data = load_and_filter(folder, start_date, end_date, file_pattern, number_of_observations)
    print(f"✅ Loaded {len(crypto_data)} cryptocurrencies\n")

    print("=" * 80)
    print("STEP 2: Computing correlation matrix")
    print("=" * 80)
    corr_matrix = compute_correlation_matrix(crypto_data)
    print(f"✅ Correlation matrix computed\n")

    print("=" * 80)
    print("STEP 3: Finding high-correlation pairs")
    print("=" * 80)
    high_corr_pairs = find_high_correlation_pairs(crypto_data, corr_matrix, corr_threshold)
    print(f"✅ Found {len(high_corr_pairs)} high-correlation pairs\n")

    print("=" * 80)
    print("STEP 4: Running rolling cointegration tests")
    print("=" * 80)
    coint_df = prepare_all_pairs(crypto_data, high_corr_pairs, window, adf_pval)
    print(f"✅ Cointegration tests completed\n")

    print("=" * 80)
    print("STEP 5: Selecting top pairs per window")
    print("=" * 80)
    top_pairs_per_window = select_top_pairs_per_window(coint_df, top_k)
    pair_counter = count_pair_occurrences(top_pairs_per_window)
    print(f"✅ Selected top {top_k} pairs for {len(top_pairs_per_window)} windows")
    print(f"✅ Found {len(pair_counter)} unique pairs across all windows\n")

    print("=" * 80)
    print("STEP 6: Building feature matrix")
    print("=" * 80)
    full_features_df = build_full_features(crypto_data, top_pairs_per_window)
    print(f"✅ Feature matrix shape: {full_features_df.shape}\n")

    if output_file:
        full_features_df.to_csv(output_file)
        print(f"✅ Saved features to {output_file}\n")

    return (crypto_data, corr_matrix, high_corr_pairs, 
            coint_df, top_pairs_per_window, full_features_df)


# ============================================================================
# 7. UTILITY FUNCTIONS
# ============================================================================

def save_cointegration_results(coint_df, output_dir):
    """
    Save cointegration results for each pair to CSV.
    
    Args:
        coint_df (dict): {(sym1, sym2): cointegration_results_df}
        output_dir (str): Directory to save files
    """
    for (sym1, sym2), df in coint_df.items():
        filename = os.path.join(output_dir, f"{sym1}_{sym2}_cointegration.csv")
        df.to_csv(filename, index=False)
        print(f"Saved {filename}")


def get_unique_symbols(top_pairs_per_window):
    """
    Get all unique symbols appearing in top pairs across all windows.
    
    Args:
        top_pairs_per_window (dict): Output from select_top_pairs_per_window
    
    Returns:
        set: Unique symbols
    """
    unique_symbols = set()
    for pairs in top_pairs_per_window.values():
        for pair_info in pairs:
            sym1, sym2 = pair_info[0]
            unique_symbols.add(sym1)
            unique_symbols.add(sym2)
    return unique_symbols


if __name__ == "__main__":
    # Example usage
    crypto_data, corr_matrix, high_corr_pairs, coint_df, top_pairs_per_window, features = (
        run_pair_selection_pipeline(
            folder="",
            start_date="2025-03-22 12:00:00", # there are data starting from 2025-03-22 10:51 for all
            end_date="2025-08-22 23:59:00",
            number_of_observations=221040,
            corr_threshold=0.85,
            window=4320,
            adf_pval=0.05,
            top_k=5,
            file_pattern="*_1m_ohlcv.parquet",
            output_file="HL_historical_pairs_features.csv"
        )
    )


STEP 1: Loading and filtering data
Loaded AAVE, 221040 rows
Loaded ADA, 221040 rows
Loaded APT, 221040 rows
Loaded ARB, 221040 rows
Loaded ATOM, 221040 rows
Loaded AVAX, 221040 rows
Loaded BCH, 221040 rows
Loaded BNB, 221040 rows
Loaded BTC, 221040 rows
Loaded DOGE, 221040 rows
Loaded DOT, 221040 rows
Loaded ENA, 221040 rows
Loaded ETC, 221040 rows
Loaded ETH, 221040 rows
Loaded HBAR, 221040 rows
Loaded LINK, 221040 rows
Loaded LTC, 221040 rows
Loaded NEAR, 221040 rows
Loaded SOL, 221040 rows
Loaded SUI, 221040 rows
Loaded TON, 221040 rows
Loaded TRX, 221040 rows
Loaded UNI, 221040 rows
Loaded WLD, 221040 rows
Loaded XLM, 221040 rows
Loaded XRP, 221040 rows
✅ Loaded 26 cryptocurrencies

STEP 2: Computing correlation matrix
✅ Correlation matrix computed

STEP 3: Finding high-correlation pairs
High correlation: AAVE & BCH = 0.89
High correlation: AAVE & BTC = 0.95
High correlation: AAVE & ETH = 0.89
High correlation: ADA & ARB = 0.91
High correlation: ADA & AVAX = 0.93
High correlation: 

C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\3314606146.py:323: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[col] = np.nan
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\3314606146.py:323: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[col] = np.nan
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\3314606146.py:323: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

✅ Feature matrix shape: (221040, 375)

✅ Saved features to HL_historical_pairs_features.csv



In [66]:
features["timestamp"]

0        2025-03-22 12:00:00
1        2025-03-22 12:01:00
2        2025-03-22 12:02:00
3        2025-03-22 12:03:00
4        2025-03-22 12:04:00
                 ...        
221035   2025-08-22 23:55:00
221036   2025-08-22 23:56:00
221037   2025-08-22 23:57:00
221038   2025-08-22 23:58:00
221039   2025-08-22 23:59:00
Name: timestamp, Length: 221040, dtype: datetime64[ns]

In [ ]:
# 24+70*5+1 (timestamp)=375

In [65]:
len(get_unique_symbols(top_pairs_per_window))

24

In [ ]:
# 70 unique cointegrated pairs; 76 which were originally with high corr

In [37]:
features

,AAVE_close,ADA_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,DOT_close,...,ETH_TRX_beta,ETH_TRX_alpha,ETH_TRX_adf_p,ETH_TRX_corr,ETC_XRP_spread,ETC_XRP_beta,ETC_XRP_alpha,ETC_XRP_adf_p,ETC_XRP_corr,timestamp
0,5.198552,-0.340956,-0.999265,1.560332,2.968721,5.783056,6.441393,11.341057,-1.779633,1.496045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-22 12:00:00
1,5.196561,-0.341617,-1.001413,1.560143,2.969594,5.782655,6.440707,11.340760,-1.780286,1.496045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-22 12:01:00
2,5.198165,-0.342335,-1.000352,1.559344,2.968156,5.781946,6.441138,11.340915,-1.779752,1.496045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-22 12:02:00
3,5.198165,-0.342335,-1.000134,1.559764,2.968618,5.782255,6.441026,11.340926,-1.779574,1.494229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-22 12:03:00
4,5.198165,-0.342885,-1.000624,1.559344,2.968978,5.781299,6.441361,11.340843,-1.780404,1.494252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-22 12:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221035,5.844153,-0.073001,-0.533122,1.573210,3.229103,6.396863,6.801628,11.669117,-1.425035,1.429354,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 23:55:00
221036,5.843863,-0.071442,-0.532867,1.572753,3.229460,6.396480,6.801672,11.669143,-1.423872,1.430287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 23:56:00
221037,5.844182,-0.071281,-0.532645,1.572546,3.229301,6.396246,6.802706,11.669339,-1.424703,1.430263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 23:57:00
221038,5.845456,-0.071281,-0.531828,1.573210,3.229934,6.396896,6.803172,11.669408,-1.424703,1.430263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 23:58:00


In [52]:
"""
Feature Engineering Pipeline for Cryptocurrency Pairs

This module provides a complete pipeline for:
1. Loading cointegration results
2. Normalizing spreads (selection vs trading windows)
3. Computing technical features (MACD, RSI, Kalman, EWMA volatility, etc.)
4. Merging funding rate data
5. Exporting final feature matrix

Usage:
    from feature_engineering_pipeline import run_full_pipeline
    
    features_df = run_full_pipeline(
        cointegration_folder="path/to/cointegration_csvs",
        input_csv="bin_futures_historical_pairs_with_spreads.csv",
        funding_data_folder="path/to/funding_data",
        output_file="final_features.csv"
    )
"""

import os
import glob
import pandas as pd
import numpy as np
import ta
from pathlib import Path


# ============================================================================
# 1. LOADING COINTEGRATION RESULTS
# ============================================================================

def load_cointegration_csvs(folder_path, files_pattern):
    """
    Load all CSV files with filenames like 'SYM1_SYM2_bin_futures_window_cointegration.csv' or any given pattern
    into a dictionary {(sym1, sym2): DataFrame}
    
    Args:
        folder_path (str): Path to folder containing cointegration CSV files
    
    Returns:
        dict: {(sym1, sym2): cointegration_results_df}
    """
    csv_files = glob.glob(os.path.join(folder_path, files_pattern))
    data_dict = {}

    for f in csv_files:
        base = os.path.basename(f).replace(".csv", "")
        try:
            # Extract symbols
            parts = base.split("_")
            sym1, sym2 = parts[0], parts[1]
        except Exception as e:
            print(f"Skipping file {f}, cannot parse symbols: {e}")
            continue

        # Read CSV
        df = pd.read_csv(f)
        data_dict[(sym1, sym2)] = df

    return data_dict


def select_top_pairs_per_window(cointegration_dict, top_k=5):
    """
    Select top-K cointegrated pairs per rolling window based on correlation.
    
    Args:
        cointegration_dict (dict): {(sym1, sym2): cointegration_results_df}
        top_k (int): Number of top pairs to keep per window (default: 5)
    
    Returns:
        dict: {(start, end): [(sym1, sym2), ...]}
    """
    top_pairs_per_window = {}
    
    # Collect all cointegrated pairs per window
    for pair, df in cointegration_dict.items():
        for _, row in df.iterrows():
            window_key = (pd.to_datetime(row["start"] + " 00:00:00" if isinstance(row["start"], str) and len(str(row["start"])) == 10 else row["start"], format="%Y-%m-%d %H:%M:%S"), 
                         pd.to_datetime(row["end"], format="%Y-%m-%d %H:%M:%S"))
            if window_key not in top_pairs_per_window:
                top_pairs_per_window[window_key] = []
            if row["cointegrated"]:
                top_pairs_per_window[window_key].append((pair, row["correlation"]))
    
    # Keep only top-K by absolute correlation
    for window_key, pairs in top_pairs_per_window.items():
        pairs.sort(key=lambda x: abs(x[1]), reverse=True)
        top_pairs_per_window[window_key] = [p[0] for p in pairs[:top_k]]
    
    return top_pairs_per_window


# ============================================================================
# 2. SPREAD NORMALIZATION
# ============================================================================

def normalize_spreads(full_df, top_pairs_per_window, folder_path="", trade_minutes=1440):
    """
    Normalize spreads for trading day using selection window beta/alpha 
    from per-pair CSVs and propagate beta/alpha/correlation to trade period.

    Parameters:
    - full_df: DataFrame with timestamp, price columns for all symbols
    - top_pairs_per_window: dict {(sel_start, sel_end): [('SYM1','SYM2'), ...]}
    - folder_path: folder containing per-pair CSVs named 
                   '{sym1}_{sym2}_bin_futures_window_cointegration.csv'
    - trade_minutes: int, duration of trading window (default 1440 = 1 day)

    Returns:
    - DataFrame with new columns:
        '{pair}_spreadNorm', '{pair}_alpha', '{pair}_beta', '{pair}_corr', '{pair}_pval'
    """
    # Drop old spread/alpha/beta/corr/adf columns
    spread_cols = [col for col in full_df.columns if ("alpha" in col) or ("beta" in col) or 
                   ("corr" in col) or ("adf" in col)]
    df = full_df.drop(columns=spread_cols, axis=1).copy()
    df = df.sort_values("timestamp").reset_index(drop=True)
    timestamps = df["timestamp"]

    for (sel_start, sel_end), top_pairs in top_pairs_per_window.items():
        # Selection window mask
        sel_mask = (timestamps >= sel_start) & (timestamps <= sel_end)
        if not any(sel_mask):
            print(f"No selection data for window {sel_start} to {sel_end}. Skipping.")
            continue

        # Trading window mask (next day after selection)
        trade_start = sel_end + pd.Timedelta(minutes=1)
        trade_end = trade_start + pd.Timedelta(minutes=trade_minutes - 1)
        trade_mask = (timestamps >= trade_start) & (timestamps <= trade_end)
        if not any(trade_mask):
            print(f"No trading data for window {sel_start} to {sel_end}. Skipping.")
            continue

        for sym1, sym2 in top_pairs:
            csv_file = os.path.join(folder_path, f"{sym1}_{sym2}_hl_futures_window_cointegration.csv")
            if not os.path.exists(csv_file):
                print(f"CSV not found for {sym1}-{sym2}. Skipping.")
                continue

            pair_df = pd.read_csv(csv_file, parse_dates=['start', 'end'])
            sel_row = pair_df[(pair_df['start'] == sel_start) & (pair_df['end'] == sel_end)]

            if sel_row.empty or not sel_row['cointegrated'].values[0]:
                continue

            alpha = sel_row['alpha'].values[0]
            beta = sel_row['beta'].values[0]
            corr = sel_row['correlation'].values[0]
            p_val = sel_row["adf_p"].values[0]

            if pd.isna(alpha) or pd.isna(beta):
                print(f"NaN alpha/beta for {sym1}-{sym2} in window {sel_start} to {sel_end}. Skipping.")
                continue

            # Compute selection window spread for mean/std
            y_sel = df.loc[sel_mask, f"{sym1}_close"]
            x_sel = df.loc[sel_mask, f"{sym2}_close"]
            sel_spread = y_sel - (alpha + beta * x_sel)
            mu = sel_spread.mean()
            sigma = sel_spread.std()
            if sigma == 0:
                sigma = 1e-9

            # Compute trading window spread and normalize
            y_trade = df.loc[trade_mask, f"{sym1}_close"]
            x_trade = df.loc[trade_mask, f"{sym2}_close"]
            if y_trade.isna().all() or x_trade.isna().all():
                print(f"All NaNs in prices for {sym1}-{sym2} in window {trade_start} to {trade_end}. Skipping.")
                continue

            trade_spread = y_trade - (alpha + beta * x_trade)
            if trade_spread.isna().any():
                print(f"NaNs in trade spread for {sym1}-{sym2} in window {trade_start} to {trade_end}")

            # Column names
            spread_norm_col = f"{sym1}_{sym2}_spreadNorm"
            alpha_col = f"{sym1}_{sym2}_alpha"
            beta_col = f"{sym1}_{sym2}_beta"
            corr_col = f"{sym1}_{sym2}_corr"
            pval_col = f"{sym1}_{sym2}_pval"

            # Store normalized spread
            df.loc[trade_mask, spread_norm_col] = (trade_spread - mu) / sigma

            # Store alpha, beta, correlation for the trade period
            df.loc[trade_mask, alpha_col] = alpha
            df.loc[trade_mask, beta_col] = beta
            df.loc[trade_mask, corr_col] = corr
            df.loc[trade_mask, pval_col] = p_val

    return df


# ============================================================================
# 3. TECHNICAL FEATURE ENGINEERING
# ============================================================================

def ewma_volatility(series, lambda_=0.94):
    """
    Exponentially weighted volatility (EWMA).
    
    Args:
        series (pd.Series): Price series
        lambda_ (float): Decay factor (default: 0.94)
    
    Returns:
        pd.Series: EWMA volatility
    """
    returns = series.pct_change().dropna()
    return returns.ewm(alpha=1 - lambda_).std()


def kalman_ma(series, window=20):
    """
    Simple rolling mean as placeholder for Kalman filter.
    
    Args:
        series (pd.Series): Price series
        window (int): Rolling window size
    
    Returns:
        pd.Series: Rolling mean
    """
    return series.rolling(window=window, min_periods=1).mean()


def compute_features_day(df_day):
    """
    Compute all features for a single day dataframe.
    Only for relevant assets and spreads (non-NA normalized spreads)
    
    Args:
        df_day (pd.DataFrame): Daily data with prices and spreads
    
    Returns:
        pd.DataFrame: Data with computed features
    """
    out = df_day.copy()
    
    # Identify spread columns for today that are not NA
    spread_cols = [col for col in df_day.columns if col.endswith("_spreadNorm") and df_day[col].notna().any()]
    
    # Identify all assets involved today
    assets = set()
    for col in spread_cols:
        parts = col.split("_")
        sym1, sym2 = parts[0], parts[1]
        assets.add(sym1)
        assets.add(sym2)
    
    price_cols = [f"{asset}_close" for asset in assets if f"{asset}_close" in df_day.columns]

    # --------------------------
    # Price-based features
    # --------------------------
    for col in price_cols:
        prices = df_day[col].astype(float)
        # MACD
        out[f"{col}Macd"] = ta.trend.MACD(prices, window_slow=14, window_fast=5).macd()
        # RSI
        out[f"{col}Rsi"] = ta.momentum.RSIIndicator(prices, window=20).rsi()
        # Kalman MA
        out[f"{col}Kalman"] = kalman_ma(prices, window=20)
        # EWMA volatility
        out[f"{col}EwmaVol"] = ewma_volatility(prices)
        # Bias (price - dynamic MA)
        out[f"{col}Bias"] = prices - out[f"{col}Kalman"]
        # Signs: ratio of positive vs negative returns over 10-min rolling window
        returns = prices.pct_change()
        out[f"{col}Signs"] = returns.rolling(window=10, min_periods=1).apply(
            lambda x: (x > 0).sum() / max(1, len(x)), raw=False
        )
        # Stochastic RSI (simplified)
        rsi = out[f"{col}Rsi"]
        out[f"{col}StochRsi"] = (rsi - rsi.rolling(20, min_periods=1).min()) / (
            rsi.rolling(20, min_periods=1).max() - rsi.rolling(20, min_periods=1).min() + 1e-12
        )
        # Upper/Lower shadow proxies
        out[f"{col}UpperShadow"] = prices - prices.rolling(2).min()
        out[f"{col}LowerShadow"] = prices.rolling(2).max() - prices
    
    # --------------------------
    # Spread-based features
    # --------------------------
    for col in spread_cols:
        s = df_day[col].astype(float)
        out[col] = s
        out[f"{col}Vol"] = ewma_volatility(s)
        out[f"{col}Kalman"] = kalman_ma(s, window=20)
        # Moving average of spread
        out[f"{col}Ma"] = s.rolling(window=20, min_periods=1).mean()

    return out


def compute_all_features(df):
    """
    Compute features day by day for all data.
    
    Args:
        df (pd.DataFrame): Full dataframe with timestamp, log prices, normalized spreads
    
    Returns:
        pd.DataFrame: Data with all computed features
    """
    df["date"] = df["timestamp"].dt.date
    feature_dfs = []
    
    for day, df_day in df.groupby("date"):
        print(f"Processing {day} ...")
        day_feats = compute_features_day(df_day)
        feature_dfs.append(day_feats)
    
    # Combine all days
    full_df = pd.concat(feature_dfs, ignore_index=True).sort_values("timestamp").reset_index(drop=True)
    
    # Drop intermediate columns
    cols_to_drop = [col for col in full_df.columns if col.endswith("_spread")]
    full_df = full_df.drop(columns=cols_to_drop, axis=1)
    
    return full_df


# ============================================================================
# 4. FUNDING RATE MERGING
# ============================================================================

def funding_interval(symbol, HL = False):
    """
    Get funding interval for a symbol.
    
    Args:
        symbol (str): Symbol name (without USDT)
    
    Returns:
        pd.Timedelta: Funding interval
    """
    if HL:
        return pd.Timedelta(hours=1)
    if symbol in ["TON", "ENA"]:
        return pd.Timedelta(hours=4)
    else:
        return pd.Timedelta(hours=8)


def merge_funding_rates(features_df, funding_data_folder, symbols=None, HL = False):
    """
    Merge funding rate data into features dataframe.
    
    Args:
        features_df (pd.DataFrame): Features dataframe with 'datetime' column
        funding_data_folder (str): Path to folder with funding rate parquet files
        symbols (list): List of symbols to merge (e.g., ['BTC', 'ETH', ...])
                       If None, uses default 25 symbols
    
    Returns:
        pd.DataFrame: Features dataframe with funding rate columns
    """
    if symbols is None:
        symbols = [
            "AAVE", "ADA", "APT", "ARB", "ATOM",
            "AVAX", "BCH", "BNB", "BTC", "DOGE",
            "DOT", "ENA", "ETC", "ETH", "HBAR",
            "LINK", "LTC", "NEAR","SOL", "SUI",
            "TON", "TRX", "UNI", "WLD", "XLM", "XRP"
        ]
    
    data_dir = Path(funding_data_folder)
    df = features_df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.sort_values("timestamp").reset_index(drop=True)

    for sym in symbols:
        try:
            path = data_dir / f"{sym}_funding_2025-03-22_2025-08-22.parquet"
            funding_df = pd.read_parquet(path)
            if "timestamp" not in funding_df.columns:
                funding_df["timestamp"] = funding_df.index
                funding_df = funding_df.reset_index(drop=True)
            funding_df["timestamp"] = pd.to_datetime(funding_df["timestamp"])
            funding_df = funding_df.sort_values("timestamp").reset_index(drop=True)

            # Compute next funding timestamp for each record
            interval = funding_interval(sym, HL)
            funding_df["next_funding_time"] = funding_df["timestamp"].shift(-1)
            funding_df.loc[funding_df["next_funding_time"].isna(), "next_funding_time"] = (
                funding_df["timestamp"] + interval
            )

            # Initialize columns
            rate_col = f"{sym}_funding"
            time_col = f"{sym}_fundingMinutesLeft"
            df[rate_col] = None
            df[time_col] = None

            for _, row in funding_df.iterrows():
                mask = (df["timestamp"] >= row["timestamp"]) & (df["timestamp"] < row["next_funding_time"])
                minutes_left = ((row["next_funding_time"] - df.loc[mask, "timestamp"]).dt.total_seconds() / 60).astype(int)
                df.loc[mask, rate_col] = row["fundingRate"]
                df.loc[mask, time_col] = minutes_left

            print(f"✅ Added {sym}: funding rate + minutes left (interval {interval.components.hours}h)")

        except Exception as e:
            print(f"⚠️ Error adding {sym}: {e}")

    print("\n🎯 Done — funding rates and countdown columns added!")
    return df


# ============================================================================
# 5. MAIN PIPELINE
# ============================================================================

def run_full_pipeline(cointegration_folder, files_pattern,
                      input_csv,
                      output_file="final_features.csv",
                      HL = False,
                      funding_data_folder="",
                      symbols=None,
                      trade_minutes=1440,
                      top_k=5,
                      sep=";"):
    """
    Complete feature engineering pipeline from cointegration to final features.
    
    Args:
        cointegration_folder (str): Path to folder with cointegration CSVs
        input_csv (str): Path to input CSV with prices and raw spreads
        output_file (str): Path to save output CSV (default: "final_features.csv")
        funding_data_folder (str): Path to folder with funding rate parquets
        symbols (list): List of crypto symbols for funding rates
        trade_minutes (int): Trading window duration in minutes (default: 1440 = 1 day)
        top_k (int): Number of top pairs to keep per window (default: 5)
        sep (str): CSV separator (default: ";")
    
    Returns:
        pd.DataFrame: Final features dataframe
    """
    print("=" * 80)
    print("STEP 1: Loading cointegration results")
    print("=" * 80)
    cointegration_dict = load_cointegration_csvs(cointegration_folder, files_pattern)
    print(f"✅ Loaded {len(cointegration_dict)} pairs\n")

    print("=" * 80)
    print("STEP 2: Selecting top pairs per window")
    print("=" * 80)
    top_pairs_per_window = select_top_pairs_per_window(cointegration_dict, top_k=top_k)
    print(f"✅ Selected top {top_k} pairs for {len(top_pairs_per_window)} windows\n")

    print("=" * 80)
    print("STEP 3: Loading input data")
    print("=" * 80)
    df = pd.read_csv(input_csv, index_col=0)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    print(f"✅ Loaded data with shape {df.shape}\n")

    print("=" * 80)
    print("STEP 4: Normalizing spreads")
    print("=" * 80)
    df_norm = normalize_spreads(df, top_pairs_per_window, folder_path=cointegration_folder, 
                                trade_minutes=trade_minutes)
    print(f"✅ Normalized spreads. Shape: {df_norm.shape}\n")

    print("=" * 80)
    print("STEP 5: Computing technical features")
    print("=" * 80)
    df_features = compute_all_features(df_norm)
    print(f"✅ Computed all features. Shape: {df_features.shape}\n")

    print("=" * 80)
    print("STEP 6: Merging funding rates")
    print("=" * 80)
    if funding_data_folder != None :
        # df_features = df_features.rename(columns={"timestamp": "datetime"})
        df_final = merge_funding_rates(df_features, funding_data_folder, symbols=symbols, HL=True)
    else:
        print("⚠️ No funding data folder provided. Skipping funding rate merge.\n")
        df_final = df_features

    print("=" * 80)
    print("STEP 7: Saving output")
    print("=" * 80)
    df_final.to_csv(output_file, index=False)
    print(f"✅ Saved final features to {output_file}\n")
    print(f"Final shape: {df_final.shape}")

    return df_final


if __name__ == "__main__":
    # Example usage
    features_df = run_full_pipeline(
        cointegration_folder="",  # Replace with your path
        files_pattern="*_hl_futures_window_cointegration.csv",
        input_csv="HL_historical_pairs_features.csv",
        output_file="HL_futures_full_features.csv",
        HL = True,
        funding_data_folder="",  # Replace with your path (optional)
        symbols=None,  # Will use default 25 symbols
        trade_minutes=1440,
        top_k=5,
        sep=";"
    )


STEP 1: Loading cointegration results
✅ Loaded 76 pairs

STEP 2: Selecting top pairs per window
✅ Selected top 5 pairs for 151 windows

STEP 3: Loading input data
✅ Loaded data with shape (221040, 375)

STEP 4: Normalizing spreads


C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:192: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[trade_mask, corr_col] = corr
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:193: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[trade_mask, pval_col] = p_val
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:187: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

✅ Normalized spreads. Shape: (221040, 445)

STEP 5: Computing technical features


C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:310: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["date"] = df["timestamp"].dt.date


Processing 2025-03-22 ...
Processing 2025-03-23 ...
Processing 2025-03-24 ...
Processing 2025-03-25 ...
Processing 2025-03-26 ...


...

Processing 2025-08-21 ...


C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:292: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Vol"] = ewma_volatility(s)
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:293: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}Kalman"] = kalman_ma(s, window=20)
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:295: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

Processing 2025-08-22 ...


C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:213: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = series.pct_change().dropna()
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:213: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = series.pct_change().dropna()
C:\Users\isaen\AppData\Local\Temp\ipykernel_19172\1222947135.py:292: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a

✅ Computed all features. Shape: (221040, 802)

STEP 6: Merging funding rates
✅ Added AAVE: funding rate + minutes left (interval 1h)
✅ Added ADA: funding rate + minutes left (interval 1h)
✅ Added APT: funding rate + minutes left (interval 1h)
✅ Added ARB: funding rate + minutes left (interval 1h)
✅ Added ATOM: funding rate + minutes left (interval 1h)
✅ Added AVAX: funding rate + minutes left (interval 1h)
✅ Added BCH: funding rate + minutes left (interval 1h)
✅ Added BNB: funding rate + minutes left (interval 1h)
✅ Added BTC: funding rate + minutes left (interval 1h)
✅ Added DOGE: funding rate + minutes left (interval 1h)
✅ Added DOT: funding rate + minutes left (interval 1h)
✅ Added ENA: funding rate + minutes left (interval 1h)
✅ Added ETC: funding rate + minutes left (interval 1h)
✅ Added ETH: funding rate + minutes left (interval 1h)
✅ Added HBAR: funding rate + minutes left (interval 1h)
✅ Added LINK: funding rate + minutes left (interval 1h)
✅ Added LTC: funding rate + minutes l

In [ ]:
24*10+70*8+24*2+2 
# probably two additional symbols were considered for fundng rate features - TON and SUI

850

In [ ]:
# something is wrong with funding

In [ ]:
# no funding rates for last day

In [81]:
features_df["BTC_funding"][219600:].isna().sum() == features_df["BTC_funding"].isna().sum()

np.True_

In [79]:
221039-1439    

219600

In [77]:
features_df["BTC_fundingMinutesLeft"].max()

73

In [ ]:
features_df["BTC_ETH"]

0         2025-03-22
1         2025-03-22
2         2025-03-22
3         2025-03-22
4         2025-03-22
             ...    
221035    2025-08-22
221036    2025-08-22
221037    2025-08-22
221038    2025-08-22
221039    2025-08-22
Name: date, Length: 221040, dtype: object